<a href="https://colab.research.google.com/github/Svetorus/Insurance/blob/master/ins_les_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Схема проекта

**Принципиальные моменты:**
*   У страховой компании нет времени "на подумать", решение о приеме на страхование или отказе, а также о тарифе должено быть сделано непосредственно в момент обращения клиента за полисом. Соответственно, сервис должен работать в режиме реального времени и давать ответ в течение максимум нескольких секунд.
*   Сервис скоринга строится обычно отдельным блоком, а фронт-система взаимодействует с сервисом через API (Application programming interface).
*   Запрос данных из внешних источников может производиться как на уровне фронт-системы, так и на уровне сервиса.
*   Один из возможных вариантов - Flask.
*   Передача информации производится путем обмена json-файлами.
*   Для тестирования можно использовать Postman.

  ![](https://drive.google.com/uc?export=view&id=1OAOF1M2U14UJWDmeJg2mwo-pgSwyNyzc)

## JSON

JSON (JavaScript Object Notation) - простой текстовый формат обмена данными, он основан на подмножестве языка программирования JavaScript.<br/>
Например, строка из нашего датасета выглядела бы следующим образом:<br/>
```
{
"ID": 1,
"Exposure": 0.583,
"RecordBeg": "2004-06-01",
"RecordEnd": "",
"DrivAge": 55,
"Gender": "Female",
...
}
```

## При внедрении

**При внедрении необходимо сделать:**
*   Определить формат json'а, в котором данные будут приниматься сервисом и отправляться обратно.
*   Определить ip-адрес и порт, на который будут поступать данные.
*   Создать во Flask необходимые роуты:<br/>
    `@app.route('/predict_example', method='POST')`<br/>
    `def predict_example():`
*   Перенести во Flask все функции преобразования данных,
    *   формат данные, приходящих от фронт-системы, может отличаться от формата исторических данных, использовавшихся при построении модели; в результате преобразований данные на вход модели должны поступить ровно в том виде, в каком была обучена модель.
*   Загрузить обученные модели.
*   Настроить логирование, запись котировок.

**Особенности:**
*   Библиотека H2O использует виртуальную Java-машину:
    *   ее нужно инициализировать один раз, а не поднимать заново для каждого расчета;
    *   вручную выделить под нее отдельный порт и указать размер используемой памяти;
    *   не создавать каждый раз заново H2O-Frame, а записывать в единожды подготовленный.
*   Необходимо удостовериться, что на всех этапах сервис отрабатывает корректно; например, можно иметь заготовленный массив котировок с заранее известными ответами.
*   Необходимо провести нагрузочное тестирование и удостовериться, что сервис справляется с нагрузкой.
*   Для согласованности версий Python, Java при переносе на другие серверы имеет смысл использовать докеры.



## Requirements

h2o version **'3.28.0.1'**

pandas version **'0.24.2'**

flask version **'1.1.1'**

sqlite3 **'3.29.0'**

## Run service

In [0]:
%run ./flask_ins.py

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,7 mins 35 secs
H2O cluster timezone:,+03:00
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.1
H2O cluster version age:,12 days
H2O cluster name:,H2O_from_python_TOSHIBA_ck4zcl
H2O cluster total nodes:,1
H2O cluster free memory:,999 Mb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


 * Serving Flask app "flask_ins" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://6d4dfcf9.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
Parse progress: |█████████████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


INFO:werkzeug:127.0.0.1 - - [29/Dec/2019 12:42:28] "POST /predict HTTP/1.1" 200 -


Parse progress: |█████████████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


INFO:werkzeug:127.0.0.1 - - [29/Dec/2019 12:43:06] "POST /predict HTTP/1.1" 200 -


## Вывод данных из БД

In [0]:
with sqlite3.connect('ins.db') as conn:
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM ins ORDER BY now DESC')
    rows = cursor.fetchall()
    conn.commit()

In [0]:
from collections import namedtuple

Result = namedtuple('Result', ['now', 'ID', 'value_Poisson', 'value_Gamma', 'value_BurningCost'])
for row in rows:
    pprint(Result(*row))

Result(now='2019-12-29 12:43:06.984526', ID='1', value_Poisson='0.22236506616314888', value_Gamma='881.8645779723482', value_BurningCost='196.09587522775857')
Result(now='2019-12-29 12:42:28.513326', ID='1', value_Poisson='0.22236506616314888', value_Gamma='881.8645779723482', value_BurningCost='196.09587522775857')
Result(now='2019-12-29 12:39:08.992914', ID='1', value_Poisson='0.22236506616314888', value_Gamma='881.8645779723482', value_BurningCost='196.09587522775857')
Result(now='2019-12-29 12:36:32.808980', ID='1', value_Poisson='0.21227993533983647', value_Gamma='1066.3756666548131', value_BurningCost='226.37015756545873')
